#검증세트

모델을 만드는 과정에서 테스트세트로 테스트를 하는 일이 많아질수록 모델은 일반화 성능이 좋은 모델이 아닌 테스트세트에 성능이 좋은 모델이 만들어진다.

즉, 테스트세트는 마지막에 한번 사용하는것이 좋다.

그렇다면 모델 테스트는 어떻게 할 것인가?

그래서 우린 검증세트도 만든다!

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine.drop('class', axis = 1).to_numpy()
target = wine['class'].to_numpy()

In [3]:
#훈련세트와 테스트세트를 먼저 나눈다.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [4]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# K-fold cross validation
휸련세트를 몇부분으로 나눠서 교차로 훈련,검증세트로 사용하는 방법


In [5]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00849915, 0.0076468 , 0.00780678, 0.00782967, 0.00802732]), 'score_time': array([0.00100946, 0.00091052, 0.00081563, 0.00108075, 0.00090194]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np
print(np.mean(scores['test_score'])) #이게 모델에서 얻을 수 있는 최상의 검증점수라고 가늠 할 수 있다.

0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
#10-fold cross 하려면
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


#하이퍼파라미터 튜닝